## 1. Introduction

---
**Milestone 3**

Nama    : Muhammad Al Ghifari

Batch   : FTDS-HKC-030

<div align="justify">

Objective

Great Expectations digunakan untuk menjaga kualitas data tetap konsisten dan sesuai standard yang sudah ditetapkan, sebelum data dipakai lebih lanjut untuk analisis maupun visualisasi. Dengan adanya validasi otomatis ini, saya dapat lebih cepat menemukan masalah seperti adanya missing values, tipe data yang tidak valid, kolom yang hilang, atau distribusi data yang tidak wajar. Dengan begitu, risiko kesalahan analisis akibat data yang kurang berkualitas dapat diminimalkan.

---

## 2. Import Libraries

<div align="justify">

Sebelum melakukan Data Loading, saya terlebih dahulu memasukkan beberapa library yang akan digunakan selama proses Great Expectations ini.

In [1]:
import pandas as pd
from pathlib import Path
import great_expectations as gx
from great_expectations.core.batch import RuntimeBatchRequest

print('Versi Pandas :', pd.__version__)
print('Versi Great Expectations :', gx.__version__)

Versi Pandas : 2.1.4
Versi Great Expectations : 0.18.22


<div align="justify">

Library tersebut mencakup :

1. `pandas (pd)`: membaca & manipulasi data tabular (DataFrame).

2. `pathlib.Path`: mengelola path file/folder yang portable antar OS (mis. Path(csv).resolve()).

3. `Great Expectations (gx)`: melakukan validasi kualitas data (expectations, validator, checkpoint, Data Docs).

4. `RuntimeBatchRequest`: mengirim DataFrame in-memory ke GX (runtime) tanpa membaca file ulang.

## 3. Data Loading

<div align="justify">

Disini saya mulai dengan pembacaan data cleaned dari hasil preprocessing dan menggunakan `parse` agar kolom `shipment_date` serta `delivery_date` dapat dibaca berdasarkan format tanggal yang benar (datetime).

In [2]:
# Load dataset hasil preprocessing
csv_path = "./data/P2M3_muhammad_al_ghifari_data_cleaned.csv"
df = pd.read_csv(csv_path, parse_dates=["shipment_date", "delivery_date"])

<div align="justify">

 Setelah itu, saya membuat variabel objek Great Expectations (`get_context()`), lalu membuat datasource runtime supaya GX bisa memakai DataFrame ini langsung tanpa baca file lagi.  Saya juga mendefinisikan Expectation Suite (kumpulan aturan cek data) dengan nama project saya, lalu membungkus dataset ke `RuntimeBatchRequest` dan membuat `validator` sebagai tool yang dipakai untuk menulis aturan (expectations), menjalankannya ke data, serta menjadi acuan saat menyimpan suite dan mengeksekusi checkpoint.

In [3]:
# Load the GE context
context = gx.get_context()

# Menambah Datasource "runtime" agar GE bisa pakai DataFrame
context.add_or_update_datasource(
    name="runtime_pandas_datasource",
    class_name="Datasource",
    execution_engine={"class_name": "PandasExecutionEngine"},
    data_connectors={
        "default_runtime_data_connector_name": {
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": ["default_identifier_name"]}})

# Membuat nama Expectation Suite (kumpulan aturan cek data)
expectation_suite_name = 'P2M3_data_suite_muhammad_al_ghifari'

# Membuat/memperbarui Expectation Suite di context
context.add_or_update_expectation_suite(expectation_suite_name)

# Membungkus dataset menjadi batch runtime (akan divalidasi)
batch_request_for_validator = RuntimeBatchRequest(
    datasource_name= "runtime_pandas_datasource",
    data_connector_name= "default_runtime_data_connector_name",
    data_asset_name= "P2M3_muhammad_al_ghifari_data_cleaned", 
    runtime_parameters= {"batch_data": df}, 
    batch_identifiers= {"default_identifier_name": "id"})

# Membuat objek validator untuk menulis & menjalankan semua expectation
validator = context.get_validator(
    batch_request = batch_request_for_validator,
    expectation_suite_name = expectation_suite_name)

validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,shipment_id,origin_warehouse,destination,carrier,shipment_date,delivery_date,weight_kg,cost,status,distance_miles,transit_days
0,SH10000,Warehouse_MIA,San Francisco,UPS,2023-10-02,2023-10-04,25.7,67.46,Delivered,291,2
1,SH10001,Warehouse_MIA,Atlanta,DHL,2023-12-06,2023-12-09,38.9,268.85,Delivered,1225,3
2,SH10002,Warehouse_LA,Houston,DHL,2023-09-18,2023-09-20,37.2,74.35,Delivered,220,2
3,SH10003,Warehouse_BOS,Seattle,OnTrac,2023-01-26,2023-02-04,42.6,187.04,Delivered,1156,9
4,SH10004,Warehouse_SF,Dallas,OnTrac,2023-06-03,2023-06-06,7.9,120.01,Delivered,1017,3


<div align="justify">

## 4. Validation

Setelah seluruh komponen siap, saya menjalankan proses validasi dengan aturan berikut:

**1. To be Unique**

**2. To Be Between Min Value And Max Value**

**3. To Be In Set**

**4. To Be In Type List**

**5. To Match Regex**

**6. Not To Be Null**

**7. Table Columns To Match Ordered List**

### 4.1 To Be Unique

In [4]:
validator.expect_column_values_to_be_unique(column="shipment_id")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

<div align="justify">

Berdasarkan Hasil Validasi GX:

1. Data Quality: Nilai unexpected_count adalah 0 dan unexpected_percent juga 0.0%, menunjukkan bahwa semua data (2000 elemen) sesuai dengan ekspektasi yang telah didefinisikan. Ini adalah indikator kuat bahwa proses ETL dan pembersihan data efektif.

2. Kelengkapan Data: Nilai `missing_count` adalah 0 dan `missing_percent` juga 0.0%, yang berarti tidak ada nilai yang hilang (missing values) dalam data yang divalidasi. Ini memastikan integritas dan kelengkapan dataset.

3. Kesiapan Data: Dengan tidak adanya unexpected values dan missing values, data watchsales yang dihasilkan dari proses ETL berada dalam kondisi yang bersih dan siap sepenuhnya untuk tahap analisis lebih lanjut beserta visualisasi.

4. ETL Effectiveness: Hasil ini secara langsung memvalidasi keberhasilan project ETL. Data yang dihasilkan benar-benar "bersih" seperti yang menjadi tujuan utama.

### 4.2 To Be Between Min Value And Max Value

In [5]:
validator.expect_column_values_to_be_between("transit_days", min_value=0, max_value=15)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

<div align="justify">

Berdasarkan Hasil Validasi GX:

1. Sudah True (Sudah Sesuai Dengan Rule Yang Dibuat), Yaitu Hasil Value Pada `transit_days` telah berada didalam Min - Max Value yang sesuai.

2. Data Quality: Nilai `unexpected_count` adalah 0 dan `unexpected_percent` juga 0.0%, menunjukkan bahwa semua data (2000 elemen) sesuai dengan ekspektasi yang telah didefinisikan. Ini adalah indikator kuat bahwa proses ETL dan pembersihan data efektif.

3. Kelengkapan Data: Nilai `missing_count` adalah 0 dan `missing_percent` juga 0.0%, yang berarti tidak ada nilai yang hilang (missing values) dalam data yang divalidasi. Ini memastikan integritas dan kelengkapan dataset.

4. Kesiapan Data: Dengan tidak adanya unexpected values dan missing values, data watchsales yang dihasilkan dari proses ETL berada dalam kondisi 'Clean' dan siap sepenuhnya untuk tahap analisis mendalam serta visualisasi.

5. ETL Effectiveness: Hasil ini secara langsung memvalidasi keberhasilan project ETL. Data yang dihasilkan benar-benar bersih yang menjadi tujuan utama.

### 4.3 To Be In Set

In [6]:
allowed_status = ["Delivered", "In Transit", "Delayed", "Canceled", "Returned", "Lost"]

In [7]:
validator.expect_column_values_to_be_in_set("status", allowed_status)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

<div align="justify">

Berdasarkan Hasil Validasi GX:

1. Sudah True (Sudah Sesuai Dengan Rule Yang Dibuat), Yaitu Hasil Unique Values (`allowed_status`) Pada `status` Sudah Sesuai Terhadap Data Yang Didefinisikan.

2. Data Quality: Nilai unexpected_count adalah 0 dan unexpected_percent juga 0.0%, menunjukkan bahwa semua data (2000 elemen) sesuai dengan ekspektasi yang telah didefinisikan. Ini adalah indikator kuat bahwa proses ETL dan pembersihan data efektif.

3. Kelengkapan Data: Nilai missing_count adalah 0 dan missing_percent juga 0.0%, yang berarti tidak ada nilai yang hilang (missing values) dalam data yang divalidasi. Ini memastikan integritas dan kelengkapan dataset.

4. Kesiapan Data: Dengan tidak adanya unexpected values dan missing values, data watchsales yang dihasilkan dari proses ETL berada dalam kondisi 'Clean' dan siap sepenuhnya untuk tahap analisis mendalam serta visualisasi.

5. ETL Effectiveness: Hasil ini secara langsung memvalidasi keberhasilan project ETL. Data yang dihasilkan benar-benar bersih yang menjadi tujuan utama.

### 4.4 To Be In Type List

In [8]:
allowed_dt_types = ["datetime64[ns]", "datetime64[ns, UTC]"]

In [9]:
validator.expect_column_values_to_be_in_type_list("shipment_date", allowed_dt_types)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "datetime64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [10]:
validator.expect_column_values_to_be_in_type_list("delivery_date",  allowed_dt_types)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "datetime64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

<div align="justify">

Berdasarkan Hasilnya Visualiasi GX:

1. Evaluated expectations: 8

2. Passed: 8 (100%)

`shipment_date` & `delivery_date` terdeteksi bertipe datetime64, sehingga siap dipakai untuk analisis berbasis tanggal (filtering, perhitungan berdasarkan minggu, dsb).

### 4.5 To Match Regex

In [11]:
validator.expect_column_values_to_match_regex("shipment_id", r"^SH\d{5}$")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

<div align="justify">

Berdasarkan Hasil Validasi GX:

1. Sudah True (Sesuai Dengan Rule Yang Dibuat): Kolom `shipment_id` memenuhi pola regex `^SH\d{5}$` (prefix SH diikuti tepat 5 digit).

2. Data Quality: `unexpected_count` = 0 dan `unexpected_percent` = 0.0% dari 2000 elemen → seluruh nilai sesuai dengan ekspektasi yang didefinisikan; konsistensi ID terjaga.

3. Kelengkapan Data: `missing_count` = 0 dan `missing_percent` = 0.0% → tidak ada nilai yang hilang pada kolom ini; integritas data terjamin.

4. Kesiapan Data: Tanpa `unexpected values` dan `missing values`, `shipment_id` siap dipakai sebagai primary key/identifier untuk analisis dan visualisasi.

5. ETL Effectiveness: Hasil ini memvalidasi keberhasilan proses pembersihan/ETL dalam menstandarkan format ID pengiriman secara konsisten.

### 4.6 Not To Be Null

In [12]:
validator.expect_column_values_to_not_be_null(column="cost")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 2000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

<div align="justify">

Berdasarkan Hasil Validasi GX:

1. Sudah True (Sesuai Dengan Rule Yang Dibuat): `expect_column_values_to_not_be_null(column="cost")` terpenuhi untuk seluruh baris.

2. Data Quality: `unexpected_count` = 0 dan `unexpected_percent` = 0.0% dari 2000 elemen, artinya semua nilai telah memenuhi aturan.

3. Kelengkapan Data: Tidak ditemukan missing value/null pada `cost`, sehingga integritas dan kelengkapan kolom terjaga.

4. Kesiapan Data: Kolom `cost` siap untuk agregasi & analisis (sum/mean) tanpa perlu imputasi atau dropna.

5. ETL Effectiveness: Mengonfirmasi proses pembersihan/ETL berhasil memastikan `cost` bebas dari missing values.

### 4.7 Table Columns To Match Ordered List

In [13]:
column_list = ['shipment_id', 'origin_warehouse', 'destination', 'carrier',
       'shipment_date', 'delivery_date', 'weight_kg', 'cost', 'status',
       'distance_miles', 'transit_days']

validator.expect_table_columns_to_match_ordered_list(column_list)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "shipment_id",
      "origin_warehouse",
      "destination",
      "carrier",
      "shipment_date",
      "delivery_date",
      "weight_kg",
      "cost",
      "status",
      "distance_miles",
      "transit_days"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

<div align="justify">

Berdasarkan Hasil Validasi GX:

1. Sudah True (Sesuai Dengan Rule Yang Dibuat): Urutan dan nama kolom persis sama dengan list yang ditentukan.

2. Data Quality: `observed_value` menunjukkan struktur tabel identik dengan skema yang diharapkan, tidak ada kolom yang salah nama/berlebih/kurang.

3. Kelengkapan Data (Skema): Tidak ada kolom yang hilang; skema lengkap sesuai definisi sehingga mengurangi risiko error saat join, agregasi, atau visualisasi.

4. Kesiapan Data: Dengan skema yang konsisten, pipeline/skrip downstream aman untuk dijalankan tanpa penyesuaian kolom.

5. ETL Effectiveness: Memvalidasi bahwa proses ETL menjaga stabilitas skema (termasuk urutan kolom), menandakan kontrol perubahan yang baik.

## 5. Validator Saving

<div align="justify">

Setelah dilakukannya semua validasi, kemudian menyimpan semua expectations yang sudah dibuat ke dalam expectation suite di Great Expectations.

In [14]:
validator.save_expectation_suite(discard_failed_expectations=False)

## 6. Validator Checkpoint

In [15]:
base_dir = Path(csv_path).resolve().parent
asset_name = Path(csv_path).stem  # "P2M3_muhammad_al_ghifari_data_cleaned"

# Datasource untuk filesystem (absolute base_directory)
context.add_or_update_datasource(
    name="file_system_datasource",
    class_name="Datasource",
    execution_engine={"class_name": "PandasExecutionEngine"},
    data_connectors={
        "default_inferred_data_connector_name": {
            "class_name": "InferredAssetFilesystemDataConnector",
            "base_directory": str(base_dir),
            "default_regex": {
                "pattern": r"(.*)\.csv",
                "group_names": ["data_asset_name"],
            },
        }
    },
)

# Checkpoint
checkpoint = context.add_or_update_checkpoint(
    name="Checkpoint_P2M3",
    validations=[{
        "batch_request": {
            "datasource_name": "file_system_datasource",
            "data_connector_name": "default_inferred_data_connector_name",
            "data_asset_name": "P2M3_muhammad_al_ghifari_data_cleaned",
            "batch_spec_passthrough": {
                "reader_options": {
                    "parse_dates": ["shipment_date", "delivery_date"]
                }
            }
        },
        "expectation_suite_name": "P2M3_data_suite_muhammad_al_ghifari",
    }],
    action_list=[
        {"name": "store_validation_result", "action": {"class_name": "StoreValidationResultAction"}},
        {"name": "update_data_docs", "action": {"class_name": "UpdateDataDocsAction"}},
    ]
)
checkpoint_result = checkpoint.run()

Calculating Metrics:   0%|          | 0/34 [00:00<?, ?it/s]

In [16]:
# Menjalankan Function Checkpoint Yang Di Define Diatas
checkpoint_result

{
  "run_id": {
    "run_name": null,
    "run_time": "2025-09-01T21:37:38.151326+07:00"
  },
  "run_results": {
    "ValidationResultIdentifier::P2M3_data_suite_muhammad_al_ghifari/__none__/20250901T143738.151326Z/72ef307c40a8631ce6fd4dc2a0796f2e": {
      "validation_result": {
        "success": true,
        "results": [
          {
            "success": true,
            "expectation_config": {
              "expectation_type": "expect_column_values_to_be_unique",
              "kwargs": {
                "column": "shipment_id",
                "batch_id": "72ef307c40a8631ce6fd4dc2a0796f2e"
              },
              "meta": {}
            },
            "result": {
              "element_count": 2000,
              "unexpected_count": 0,
              "unexpected_percent": 0.0,
              "partial_unexpected_list": [],
              "missing_count": 0,
              "missing_percent": 0.0,
              "unexpected_percent_total": 0.0,
              "unexpected_percent_n

<div align="justify">

Semua hasil pemeriksaan di atas menunjukkan bahwa validasi yang telah disimpan berhasil, artinya semua validasi untuk dataset yang telah dibersihkan melalui preprocessing ini berhasil dan siap untuk dilakukannya analisis dan visualisasi lebih lanjut di Elasticsearch-Kibana.

## 7. Data Documentations

In [17]:
# Membuat dokumentasi validator
context.build_data_docs()

{'local_site': 'file:///var/folders/2j/1mq11gq56dv_55_8qm5qf_z80000gn/T/tmpehp9f_qs/index.html'}

In [18]:
# Membuka dokumentasi validator
context.open_data_docs()

## 8. Conclusion

<div align="justify">

Berdasarkan Hasil Semua Validasi GX, diantaranya:

**1. To be Unique - Success**

**2. To Be Between Min Value And Max Value - Success**

**3. To Be In Set - Success**

**4. To Be In Type List - Success**

**5. To Match Regex - Success**

**6. Not To Be Null - Success**

**7. Table Columns To Match Ordered List - Success**

Hasil dari GX Validasi ini bisa dibuktikan dengan hasil dokumen yang sudah didapat ke dalam `P2M3_muhammad_al_ghifari_GX_result.png`